# Chapter 5 정규선형모델

# 5.1 연속형 독립변수가 하나인 모델 (단순회귀) 

5.1.1 분석 준비

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
#
# %cd /content/drive/MyDrive/Colab\ Notebooks/StatisticsWithPython/code/chap05

In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import statsmodels.formula.api as smf
import statsmodels.api as sm

import warnings
warnings.filterwarnings("ignore")

# %matplotlib inline

5.1.2 데이터 읽어 들이기와 표시

In [ ]:
beer = pd.read_csv("../data/5-1-1-beer.csv")
print(beer.head())

In [ ]:
graph = sns.jointplot(x="temperature", y="beer", data=beer)

x = beer["temperature"]
y = beer["beer"]

r, p = stats.pearsonr(x, y)

phantom, = graph.ax_joint.plot([], [], linestyle="", alpha=0)
graph.ax_joint.legend([phantom],[f'r={r:f}, p={p:f}'])
plt.show()

5.1.3 모델 구축

5.1.4 statsmodels를 이용한 모델링

In [ ]:
lm_model = smf.ols(formula = "beer ~ temperature", data=beer).fit()

5.1.5 추정 결과 표시와 계수 검정

In [ ]:
print(lm_model.summary())

5.1.6 summary 함수의 출력 내용 설명

5.1.7 AIC를 이용한 모델 선택

AIC = -2 x (최대로그우도 - 추정된 파라미터의 수)

In [ ]:
null_model = smf.ols("beer ~ 1", data=beer).fit()
print(f"null model AIC: {null_model.aic}")
print(f"lm model AIC  : {lm_model.aic}")

print(f"lm model LLF  : {lm_model.llf}")
print(f"lm model df   : {lm_model.df_model}")

print(f"lm model AIC (equation) : {-2 * (lm_model.llf - (lm_model.df_model + 1))}")

5.1.8 회귀직선

- 종속변수가 연속형 변수일 경우에는 전통적으로 회귀라고 부르기 때문에 회귀라는 이름을 사용했습니다.

5.1.9 seaborn을 이용한 회귀직선 그래프 그리기

In [ ]:
# TODO confidnence interval shape
sns.lmplot(x="temperature", y="beer", data=beer)
plt.show()

In [ ]:
print(lm_model.params)
print()
print(lm_model.conf_int())
print()
print("standard errors")
print(lm_model.bse)
print()
print("standard deviation")
print(lm_model.bse * np.sqrt(len(beer['beer'])))

In [ ]:
sns.lmplot(x="temperature", y="beer", data=beer)

a_ci0 = lm_model.conf_int()[0]['temperature']
b_ci0 = lm_model.conf_int()[0]['Intercept']

a_ci1 = lm_model.conf_int()[1]['temperature']
b_ci1 = lm_model.conf_int()[1]['Intercept']

a_ci_list = [a_ci0, a_ci1]
b_ci_list = [b_ci0, b_ci1]
for a_ci in a_ci_list:
    for b_ci in b_ci_list:
        lm_predict = b_ci + a_ci * beer['temperature'] 
        plt.plot(beer['temperature'], lm_predict, 'r', alpha=0.5)

a = lm_model.params['temperature']
b = lm_model.params['Intercept']
lm_predict = b + a * beer['temperature'] 
plt.plot(beer['temperature'], lm_predict, 'k')

plt.show()

reference: 

[1] https://stats.stackexchange.com/questions/85560/shape-of-confidence-interval-for-predicted-values-in-linear-regression

[2] https://people.duke.edu/~rnau/mathreg.htm

5.1.10 모델을 이용한 예측

In [ ]:
print(lm_model.predict())
print()

In [ ]:
print("기온이 0도일 때의 맥주 매상")
print(lm_model.predict(pd.DataFrame({"temperature":[0]})))
print()

In [ ]:
print("모델 파라미터")
print(lm_model.params)
print()

In [ ]:
print("기온이 20도일 때의 맥주 매상")
print(lm_model.predict(pd.DataFrame({"temperature":[20]})))
print()

In [ ]:
print("파라미터 확인")
beta0 = lm_model.params[0]
beta1 = lm_model.params[1]
temperature = 20
print("기온이 20도일 때의 맥주 매상")
print(beta0 + beta1 * temperature)
print()

5.1.11 잔차 계산

- 정규선형모델의 경우에는 잔차가 '평균이 0인 정규분포'를 따르는 것이므로 모델이 그 분포를 따르고 있는지 체크하게 됩니다.

$$residuals = {y - {\hat{y}}}$$

In [ ]:
resid = lm_model.resid
print(resid.head(3))

In [ ]:
y_hat = beta0 + beta1 * beer.temperature
print(y_hat.head(3))

In [ ]:
print("residual")
print((beer.beer - y_hat).head(3))

5.1.12 결정계수

$$R^2 = {{\sum_{i=1}^N(\hat{y} - \mu)^2} \over {\sum_{i=1}^N(y - {\mu})^2}}$$

In [ ]:
mu = np.mean(beer.beer)
y = beer.beer
yhat = lm_model.predict()

r_squared = np.sum((yhat - mu) ** 2) / np.sum((y - mu) ** 2)
print(f"r_squared (equation 01): {r_squared}")

In [ ]:
print(f"r_squared (lm_model): {lm_model.rsquared}")

In [ ]:
print(np.sum((yhat - mu) ** 2) + sum(resid ** 2))

In [ ]:
print(np.sum((y - mu) ** 2))

$$R^2 = 1 - {{\sum_{i=1}^Nresiduals^2} \over {\sum_{i=1}^N(y - {\mu})^2}}$$

In [ ]:
r_squared = 1 - np.sum(resid ** 2) / np.sum((y - mu) ** 2)
print(f"r_squared: (equation 02): {r_squared}")

5.1.13 수정된 결정계수

- 독립변수의 수가 늘어나는 것에 대해 페널티를 적용한 **결정계수**를 수정된 결정계수라고 부릅니다.

$$R^2 = 1 - {{\sum_{i=1}^Nresiduals^2 / (N-s-1)} \over {\sum_{i=1}^N(y - {\mu})^2 / (N-1)}}$$

In [ ]:
n = len(beer.beer)
s = 1
r_squared_adj = 1 - ((np.sum(resid ** 2)) / (n - s - 1)) / (np.sum((y - mu) ** 2) / (n - 1))
print(f"r_squared_adj (equation): {r_squared_adj}")

In [ ]:
print(f"r_squared_adj (lm_model): {lm_model.rsquared_adj}")

5.1.14 잔차 그래프

In [ ]:
# sns.displot
sns.distplot(resid)

In [ ]:
# sns.displot
sns.displot(resid, stat="density", kde=True)

In [ ]:
# sns.histplot
sns.histplot(resid, stat="density", kde=True)

In [ ]:
graph = sns.jointplot(lm_model.fittedvalues, resid)

r, p = stats.pearsonr(lm_model.fittedvalues, resid)

phantom, = graph.ax_joint.plot([], [], linestyle="", alpha=0)
graph.ax_joint.legend([phantom],[f'r={r:f}, p={p:f}'])

5.1.15 Q-Q 플롯

- 이론상의 분위점과 실제 데이터의 분위점을 산포도 그래프로 그린 것을 **Q-Q 플롯**이라고 합니다.

In [ ]:
fig = sm.qqplot(resid, line="s")

In [ ]:
resid_sort = resid.sort_values()
resid_sort.head()

In [ ]:
# 이번에는 Q-Q 플롯을 직접 만들어보겠습니다.
nobs = len(resid_sort)
cdf = np.arange(1, nobs + 1) / (nobs + 1)
print(cdf)

In [ ]:
ppf = stats.norm.ppf(cdf)
print(ppf)

In [ ]:
plt.plot(ppf, resid_sort, 'bo')

참고문헌: 

[1] 파이썬으로 배우는 통계학 교과서; 바바 신야 지음, 윤옹식 옮김; 한빛미디어 (2020)